In [1]:
#Importing library

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import datetime

from scipy import stats
from scipy.stats import skew
from scipy.stats import norm
from scipy.stats.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
#Importing dataset
df = pd.read_csv("dataset/train_processed.csv")

In [3]:
df_date=df['Date']
df_no_date=df.drop(['Date'], axis=1)
df_norm= (df_no_date - df_no_date.min()) / (df_no_date.max() - df_no_date.min())
df=df_norm
df['Date']=df_date

In [4]:
df_val3=df.loc[(df['Date']>='2018-01-01')]
df_val3_rest=df.loc[(df['Date']<'2018-01-01')]
df_val2=df_val3_rest.loc[(df_val3_rest['Date']>='2017-11-01')]
df_val2_rest=df.loc[(df['Date']<'2017-11-01')]
df_val1=df_val2_rest.loc[(df_val2_rest['Date']>='2017-09-01')]
df_train=df.loc[(df['Date']<'2017-09-01')]

In [ ]:
df_train_clear=df_train.drop(['Date'], axis=1)
df_val1_clear=df_val1.drop(['Date'], axis=1)
y=df_train_clear['NumberOfSales']
y_val1=df_val1_clear['NumberOfSales']
X=df_train_clear.drop(['StoreID','NumberOfSales','NumberOfCustomers'], axis=1)
X_val1=df_val1_clear.drop(['StoreID','NumberOfSales','NumberOfCustomers'], axis=1)

In [ ]:
nt=201
na=12
depth=21

feature_names = list(X.columns)

forest = RandomForestRegressor(max_depth=depth, random_state=0, n_estimators=nt, max_features=na, n_jobs=-1)
forest.fit(X, y)
y_pred=forest.predict(X_val1)
mse_val1=mean_squared_error(y_val1,y_pred)
mae_val1=mean_absolute_error(y_val1,y_pred)
print("con tutte le feature", sqrt(mse_val1), mae_val1)

for feature in feature_names[:21]:
    X=df_train_clear.drop(['StoreID','NumberOfSales','NumberOfCustomers'], axis=1)
    X_val1=df_val1_clear.drop(['StoreID','NumberOfSales','NumberOfCustomers'], axis=1)
    X=X.drop([feature], axis=1)
    X_val1=X_val1.drop([feature], axis=1)
    forest = RandomForestRegressor(max_depth=depth, random_state=0, n_estimators=nt, max_features=na, n_jobs=-1)
    forest.fit(X, y)
    y_pred=forest.predict(X_val1)
    mse_val1=mean_squared_error(y_val1,y_pred)
    mae_val1=mean_absolute_error(y_val1,y_pred)
    print(feature, sqrt(mse_val1), mae_val1)